In [1]:
import torch
from os import listdir, path
from PIL import Image
import pandas as pd
import torchvision.transforms.functional
from torch.utils.data import DataLoader
import os
import json
from PIL import Image
from torch.utils.data import Dataset

class TRANSFORM(object):
    def __call__(self, image, target):
        image = torchvision.transforms.functional.to_tensor(image)
        image = torchvision.transforms.functional.normalize(image, mean=[0.485, 0.456, 0.406],
                                                            std=[0.229, 0.224, 0.225])
        return image, target

In [94]:
xml_paths = 'C:/Users/USER/PycharmProjects/UniWork/MinivanDetection/Dataset/van/pascal'
img_dir = 'C:/Users/USER/PycharmProjects/UniWork/MinivanDetection/Dataset/van/images'
LABELS = {'van': 1}

In [36]:
import os
import xml.etree.ElementTree as ET
from PIL import Image
from torch.utils.data import Dataset
from os import listdir, path
class CocoDataset(Dataset):
    def __init__(self, xml_paths, img_dir, transforms = None):
        self.xml_paths = xml_paths
        self.xml_files = list(sorted(listdir(path.join(xml_paths))))
        self.img_dir = img_dir
        self.images = list(sorted(listdir(path.join(img_dir))))
        self.transforms = transforms
        self.data = []



    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        xml_path = path.join(self.xml_paths, self.xml_files[idx])
        img_path = path.join(self.img_dir, self.images[idx])
        img = Image.open(img_path).convert("RGB")

        tree = ET.parse(xml_path)
        root = tree.getroot()
        boxes , area, labels = [] , [], []
        for obj in root.findall('object'):
                name = obj.find('name').text
                bndbox = obj.find('bndbox')
                xmin = int(bndbox.find('xmin').text)
                ymin = int(bndbox.find('ymin').text)
                xmax = int(bndbox.find('xmax').text)
                ymax = int(bndbox.find('ymax').text)
                boxes.append([xmin, ymin, xmax, ymax])
                area.append((ymax - ymin) * (xmax - xmin))
                labels.append(LABELS[name])

        boxes = torch.as_tensor(boxes, dtype=torch.float)
        labels = torch.as_tensor(labels, dtype=torch.int64)
        area = torch.as_tensor(area, dtype=torch.float32)
        image_id = torch.tensor([idx])
        iscrowd = torch.zeros((1,), dtype=torch.int64)
        target = {"boxes": boxes, "labels": labels, "image_id": image_id, "area": area, "iscrowd": iscrowd}

        if self.transforms is not None:
            img, target = self.transforms(img, target)

        return img, target

In [35]:
dataset = CocoDataset('C:/Users/USER/PycharmProjects/UniWork/MinivanDetection/Dataset/van/pascal', 'C:/Users/USER/PycharmProjects/UniWork/MinivanDetection/Dataset/van/images' )

In [55]:
data = []

xml_files = list(sorted(listdir(path.join(xml_paths))))
images = list(sorted(listdir(path.join(img_dir))))


In [69]:

for idx in range(10):
    xml_path = path.join(xml_paths, xml_files[idx])
    img_path = path.join(img_dir, images[idx])
    tree = ET.parse(xml_path)
    root = tree.getroot()
    boxes = []
    for obj in root.findall('object'):
            name = obj.find('name').text

            bndbox = obj.find('bndbox')
            xmin = int(bndbox.find('xmin').text)
            ymin = int(bndbox.find('ymin').text)
            xmax = int(bndbox.find('xmax').text)
            ymax = int(bndbox.find('ymax').text)
            boxes.append([xmin, ymin, xmax, ymax])
            data.append((img_path, (xmin, ymin, xmax, ymax), name))

In [70]:
boxes

[[152, 288, 798, 613]]

In [100]:
xml_path = 'C:/Users/USER/PycharmProjects/UniWork/MinivanDetection/Dataset/van/pascal/000fe628fd1fa9d2.xml'#path.join(xml_paths, xml_files[idx])
img_path = 'C:/Users/USER/PycharmProjects/UniWork/MinivanDetection/Dataset/van/images/000fe628fd1fa9d2.jpg'#path.join(img_dir, images[idx])
tree = ET.parse(xml_path)
root = tree.getroot()
boxes = []
area = []
labels = []
for obj in root.findall('object'):
        name = obj.find('name').text
        bndbox = obj.find('bndbox')
        xmin = int(bndbox.find('xmin').text)
        ymin = int(bndbox.find('ymin').text)
        xmax = int(bndbox.find('xmax').text)
        ymax = int(bndbox.find('ymax').text)
        boxes.append([xmin, ymin, xmax, ymax])
        area.append((ymax - ymin) * (xmax - xmin))
        labels.append(LABELS[name])



1
1


In [97]:
labels

[1, 1]

In [98]:
l = torch.as_tensor(labels, dtype=torch.int64)

In [102]:
l.shape

torch.Size([2])

In [90]:
a = torch.as_tensor(boxes, dtype=torch.float)

In [91]:
a.shape

torch.Size([2, 4])

In [92]:
b = torch.as_tensor(area, dtype=torch.float)

In [93]:
b.shape

torch.Size([2])

In [101]:
area

[27456, 20043]